# Visualization based on pyvista

In [1]:
import openseespy.opensees as ops
import opstool as opst

Load the 3D Arch Bridge finite element model.

In [2]:
opst.load_ops_examples("ArchBridge")

## Display OpenSeesPy Geometry Model

In [3]:
ModelData = opst.GetFEMdata(results_dir="opstool_output")
ModelData.get_model_data(save_file="ModelData.hdf5")

Model data saved in opstool_output/ModelData.hdf5!

In [4]:
opsvis = opst.OpsVisPyvista(point_size=2, line_width=3,
                            colors_dict=None, theme="document",
                            color_map="Spectral", on_notebook=False,
                            results_dir="opstool_output")

In [5]:
opsvis.model_vis(input_file="ModelData.hdf5",
                 show_node_label=False, show_ele_label=False,
                 show_local_crd=True, label_size=8,
                 show_outline=True,
                 opacity=1.0,
                 save_fig='images/ModelVis.svg')

## Display Eigen Analysis

<div class="alert alert-info">

Note

Before performing the eigenvalue analysis, you need to ensure that the OpenSeesPy model is correct and that the mass is set.

</div>

Obtain the first 15 orders of modal data.

In [6]:
ModelData.get_eigen_data(mode_tag=20, solver="-genBandArpack",
                         save_file='EigenData.hdf5')

Eigen data saved in opstool_output/EigenData.hdf5 !

In [7]:
opsvis.eigen_vis(input_file='EigenData.hdf5',
                 mode_tags=[1, 20], subplots=False,
                 alpha=None, show_outline=False,
                 show_origin=False, opacity=1.0,
                 show_face_line=False, save_fig='images/EigenVis.svg')

Of course, *subplots* set to True will return a multi-sub plot.

In [8]:
opsvis.eigen_vis(input_file='EigenData.hdf5',
                 mode_tags=[1, 16], subplots=True,
                 alpha=None, show_outline=False,
                 show_origin=False, opacity=1.0,
                 show_face_line=False, save_fig='images/EigenVis2.svg')

## Display Response Data

In [9]:
opst.gen_grav_load(ts_tag=1, pattern_tag=1,
                   factor=-9.81, direction="Z")

In [10]:
Nsteps = 10
ops.wipeAnalysis()
ops.system('BandGeneral')
ops.constraints('Transformation')
ops.numberer('RCM')
ops.test('NormDispIncr', 1.0e-12, 10, 3)
ops.algorithm('Newton')
ops.integrator('LoadControl', 1 / Nsteps)
ops.analysis('Static')

# Important!!!! to clear step data in previous analysis case!
ModelData.reset_steps_state()
# start analysis loop
for i in range(Nsteps):
    ok = ops.analyze(1)
    ModelData.get_node_resp_step(num_steps=Nsteps,
                                 total_time=10000000000,
                                 stop_cond=False,
                                 save_file="NodeRespStepData-1.hdf5",
                                 model_update=False)
    ModelData.get_frame_resp_step(num_steps=Nsteps,
                                  total_time=10000000000,
                                  stop_cond=False,
                                  save_file="BeamRespStepData-1.hdf5")

Node response data saved in opstool_output/NodeRespStepData-1.hdf5 !

Frame elements response data saved in opstool_output/BeamRespStepData-1.hdf5 !

In [11]:
opsvis.deform_vis(input_file="NodeRespStepData-1.hdf5",
                  slider=True,
                  response="disp", alpha=None,
                  show_outline=False, show_origin=True,
                  show_face_line=False, opacity=1,
                  save_fig="images/DefoVis.svg",
                  model_update=False)

### Display Frame Element Response

In [12]:
opsvis.frame_resp_vis(input_file="BeamRespStepData-1.hdf5",
                      ele_tags=None,
                      slider=True,
                      response="My",
                      show_values=False,
                      alpha=None,
                      opacity=1,
                      save_fig="images/FrameRespVis.svg")